# Incompetent Teacher Unlearning

https://arxiv.org/abs/2205.08096 

In [ ]:
import copy
import json
import sys

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchinfo import summary
from torchvision import transforms
from tqdm import tqdm

In [ ]:
drive = None
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
path = "./"
sys.path.append(path)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
path = path if drive is None else "/content/drive/MyDrive/self-learn/unlearning"

In [ ]:
from constants import *
from utils import set_seed, train_data, val_data, \
                    train_loader, val_loader, fine_labels
from models import get_model, get_attack_model
    
set_seed()

In [ ]:
model, optimizer = get_model(SEED)

# Setup

In [1]:
target_class = 23
fine_labels[target_class]

NameError: name 'fine_labels' is not defined

In [33]:
## TODO:


## Train as usual a competent teacher on the whole dataset, and initialize a student using its params
## Create incompetent teacher via randomly initialized params

## use unlearn loss to train student
## for comparison, train a model entirely on retrain and see what accuracies / ZRF it achieves

In [31]:
def unlearn_loss(s_logits, ct_logits, it_logits, labels):
    
    ct_probs, it_probs = F.softmax(ct_logits, dim=-1), F.softmax(it_logits, dim=-1)
    # assuming 1 = forget
    teacher_out = labels * it_probs + (1-labels) * ct_probs
    s_log_probs = F.log_softmax(s_logits, dim=-1) # F.kl_div expects log softmax in first arg
    
    return F.kl_div(s_log_probs, teacher_out)

In [32]:
def JSDiv(model_1_logits, model_2_logits):
    model_1_probs, model_2_probs = F.softmax(model_1_logits, dim=-1), F.softmax(model_2_logits, dim=-1)
    m = (model_1_probs + model_2_probs) / 2
    return (F.kl_div(torch.log(model_1_probs), m) + F.kl_div(torch.log(model_2_probs), m)) / 2

def ZRF(model_1, model_2, forget_loader):
    # TODO: loop over all batches in forget_loader and concatenate model logits
    pass
    ## to validate
    # 1-(JSDiv() / len(forget_loader))